# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
IMG_DIR = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
VAL_PCT = 20
VAL_FORMAT = 'png'

In [5]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
print(os.getcwd())

In [6]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [7]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [8]:
inputs_shape= IMG_SIZE=(200, 200, 3)
BATCH_SIZE = 32
latent_dim = 32
intermediate_dim = 32

In [9]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [10]:
import tensorflow as tf

In [11]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

In [12]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [13]:
_instance_scale

1.0

In [14]:
import numpy as np
from collections.abc import Iterable

In [15]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [16]:
_outputs_shape

120000

# Model's Layers definition

In [17]:
units=intermediate_dim
c=50
menc_lays = [
    tf.keras.layers.Conv2D(filters=units, kernel_size=1, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latent_dim)
]

venc_lays = [
    tf.keras.layers.Conv2D(filters=units, kernel_size=1, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latent_dim)
]

dec_lays = [
    tf.keras.layers.Dense(units=units*c*c, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(c , c, units)),
    tf.keras.layers.Conv2DTranspose(filters=units*3, kernel_size=3, strides=(4, 4), padding="SAME", activation='relu'),
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, strides=(1, 1), padding="SAME")
]

# Model definition

In [19]:
model_name = dataset_name+'ConvILTVAAEpsnr'
recording_dir='recording'+sep_local+model_name

In [22]:
from training.adversarial_basic.generative_adversarial.autoencoders.VAAE import VAAE as AE

In [23]:
inputs_shape=IMG_SIZE

In [24]:
variables_params = \
[
    {
        'name': 'inference_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': menc_lays
    }
    ,
    {
        'name': 'inference_logvariance', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': venc_lays
    }
    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [25]:
from utils.data_and_files.file_utils import create_if_not_exist

In [26]:
_restore = os.path.join(recording_dir, 'var_save_dir')

In [27]:
create_if_not_exist(_restore)
_restore

'recording\\pokemonConvIVAAEpsnr\\var_save_dir'

In [28]:
#to restore trained model, set filepath=_restore

In [ ]:
from statistical.basic_adversarial_losses import \
    create_inference_discriminator_real_losses, \
    create_inference_discriminator_fake_losses, \
    create_inference_generator_fake_losses, \
    create_genertive_discriminator_real_losses, \
    create_genertive_discriminator_fake_losses, \
    create_genertive_generator_fake_losses, \

inputs_latent_discriminator_losses = {
    'inference_discriminator_real_outputs': create_inference_discriminator_real_losses,
    'inference_discriminator_fake_outputs': create_inference_discriminator_fake_losses,
    'inference_generator_fake_outputs': create_inference_generator_fake_losses,
    ''generative_discriminator_fake_outputs'': create_genertive_discriminator_real_losses,
    ''generative_discriminator_fake_outputs'': create_genertive_discriminator_fake_losses,
    'genertive_generator_fake_outputs': create_genertive_generator_fake_losses,
}

In [29]:
ae = AE( 
    adversarial_losses=inputs_latent_discriminator_losses,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference_mean"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_mean_inputs (InputLa [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 100, 100, 32)      128       
_________________________________________________________________
flatten (Flatten)            (None, 320000)            0         
_________________________________________________________________
dense (Dense)                (None, 32)                10240032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
activity_regularization (Act (None, 32)           

  WARNING  | None


Model: "inference_logvariance"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_logvariance_inputs (Input [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      128       
_________________________________________________________________
flatten_1 (Flatten)          (None, 320000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                10240032  
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
activity_regularization_1 (A (None, 32

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 80000)             2640000   
_________________________________________________________________
reshape (Reshape)            (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 200, 200, 96)      27744     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 200, 200, 3)       2595      
_________________________________________________________________
batch_normalization_2 (Batch (None, 200, 200, 3)       12        
_________________________________________________________________
dropout_2 (Dropout)          (None, 200, 200, 3)       0

  WARNING  | None


In [30]:
from evaluation.quantitive_metrics.psnr import psnr


In [31]:
ae.compile(loss={'x_logits': psnr})

Model: "pokemonConvIVAAEpsnr"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inference_logvariance_inputs (InputLay [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
inference_logvariance (Model)          (None, 32)           10240288    inference_logvariance_inputs[0][0]      
__________________________________________________________________________________________________
tf_op_layer_mul (TensorFlowOpLa [(None, 32)]         0           inference_logvariance[1][0]             
__________________________________________________________________________________________________
tf_op_layer_Exp (TensorFlowOpLa [(None, 32)]         0           tf_op_layer_mul[0][0]            
___________________________________________________

In [32]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [33]:
progbar = NotebookPrograssBar(leave_outer=False)

In [34]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [35]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [36]:
csv_dir = os.path.join(recording_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'recording\\pokemonConvIVAAEpsnr\\csv_dir\\pokemonConvIVAAEpsnr.csv'

In [37]:
image_gen_dir = os.path.join(recording_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [38]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [39]:
import numpy as np

In [ ]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=750,
    epochs=int(1e6), 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=250
)